### Shared Word

In [81]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [82]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [130]:
lang_folder = "Turkish"  # Primary language
lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [131]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [132]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_French (fra)_Etymologeek_Result_All.xlsx']

In [133]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,polis,polis,Turkish,(Noun) Police (an organisation that enforces t...,police,French (fra),(Quebec) cop (police officer). (typography) fo...
1,dün,dünit,Turkish,(Noun) (mineralogy) dunite.(mineralogy) dunite.,dunite,French (fra),NaN
2,alo,alo,Turkish,(Interjection) Hello (only when picking up the...,allo,French (fra),NaN
3,komik,komik,Turkish,(Adjective) Funny.Funny.,comique,French (fra),Comic Clown (person who acts silly). Comedy.
4,telefon,telefon,Turkish,(Noun) Telephone.Telephone.,téléphoner,French (fra),"(intransitive) To call, to ring, to telephone."
...,...,...,...,...,...,...,...
203,papy,papyon,Turkish,(Noun) Bowtie.Bowtie.,papillon,French (fra),(swimming) butterfly stroke. Butterfly. KnotAl...
204,jambon,jambon,Turkish,(Noun) Ham.Ham.,jambe,French (fra),Leg.
205,entri,entrika,Turkish,(Noun) Intrigue.Intrigue.,intriguer,French (fra),"(intransitive) to scheme, to connive. (transit..."
206,motori,motorin,Turkish,(Noun) (chemistry) diesel (fuel).(chemistry) d...,motorine,French (fra),NaN


In [134]:
word_list = df_pair["dict_entry_main"].values.tolist()

In [163]:
# Select N Gram Type
sent_ngram_file = "Sentence" # Two, Three, Four, Five, Sentence
 
if sent_ngram_file.lower() != "sentence":
    data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
    folder_path = "N Gram"
else:
    data_kind = "sentence"
    folder_path = "Sentence"

In [164]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
#df = df.head(100000)
df

,sentence,frequency,code
0,evet,1948596,G1-L4-F1948596-0
1,fikret,1533918,G1-L6-F1533918-1
2,hayır,1250401,G1-L5-F1250401-2
3,tamam,882921,G1-L5-F882921-3
4,ne,753105,G1-L2-F753105-4
...,...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6,G6-L35-F6-2913965
2913966,eminim uydurmuştur,6,G2-L18-F6-2913966
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6,G9-L49-F6-2913967
2913968,tabii gerçek bir profesyonel o,6,G5-L30-F6-2913968


In [165]:
d_list  = df.iloc[:,0].values.tolist()

In [166]:
resultlist = []

In [167]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [168]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
df_merge_result.reset_index(drop=True, inplace=True)
df_merge_result

,sentence,frequency,code
0,alo polis,96,G2-L9-F96-65399
1,pardon şef,54,G2-L10-F54-121723
2,pardon madam,44,G2-L12-F44-153252
3,süper polis,26,G2-L11-F26-303193
4,sivil polis,26,G2-L11-F26-305286
5,şef garson,25,G2-L10-F25-315057
6,garson şampanya,25,G2-L15-F25-320974
7,romantik müzik,23,G2-L14-F23-356034
8,şampanya madam,17,G2-L14-F17-573643
9,kod gri,15,G2-L7-F15-673605


In [169]:
df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)

#### Concat Result

In [170]:
df_word = df_pair.loc[:,["dict_entry_main"]]
df_word.rename(columns={"dict_entry_main":"word"}, inplace=True)
df_word

,word
0,polis
1,dünit
2,alo
3,komik
4,telefon
...,...
203,papyon
204,jambon
205,entrika
206,motorin


In [171]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,nükleer füze,153
1,sivil polis,145
2,alo polis,128
3,kongre onur,107
4,pozitif enerji,91
...,...,...
140,grup polis,6
141,amatör füze,6
142,atom şampiyon,6
143,nükleer şok,6


In [172]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,pozitif enerji pozitif,14
1,pardon pardon pozitif,14


In [173]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram


In [174]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram


In [175]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,alo polis,96
1,pardon şef,54
2,pardon madam,44
3,süper polis,26
4,sivil polis,26
5,şef garson,25
6,garson şampanya,25
7,romantik müzik,23
8,şampanya madam,17
9,kod gri,15


In [176]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,polis,nükleer füze,153.0,pozitif enerji pozitif,14.0,NaN,NaN,NaN,NaN,alo polis,96.0
1,dünit,sivil polis,145.0,pardon pardon pozitif,14.0,NaN,NaN,NaN,NaN,pardon şef,54.0
2,alo,alo polis,128.0,NaN,NaN,NaN,NaN,NaN,NaN,pardon madam,44.0
3,komik,kongre onur,107.0,NaN,NaN,NaN,NaN,NaN,NaN,süper polis,26.0
4,telefon,pozitif enerji,91.0,NaN,NaN,NaN,NaN,NaN,NaN,sivil polis,26.0
...,...,...,...,...,...,...,...,...,...,...,...
203,papyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,jambon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,entrika,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,motorin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result.xlsx", index=False)